In [9]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics.pairwise import euclidean_distances
from scipy import sparse

from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

from ipywidgets import interact, widgets
from tkinter import *


In [10]:
pd.set_option("display.max_colwidth", -1)
sample_df=pd.read_excel("test_data.xlsx").fillna(0)

C:\Users\iPric\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [11]:
tf = TfidfVectorizer(
    analyzer='word', 
    ngram_range=(1, 2), 
    min_df=0.03,
    max_features=5000,
  #token_pattern=r'\b[^\d\W]+\b',
    stop_words='english',
    sublinear_tf=True
)

#recipe_ids=sample_df['id'].tolist()

tfidf_matrix = tf.fit_transform(sample_df['ingredients'])

In [12]:
#Cosine Similarity

cosine_similarities = linear_kernel(tfidf_matrix,tfidf_matrix) 

indices= pd.Series(sample_df.index, index=sample_df['id'])

In [20]:

##### FInal recommended recipes

def get_recommendations_cosine(id, num, cosine_similarities=cosine_similarities):
    idx = indices[id]
    
    sim_scores =list(enumerate(cosine_similarities[idx]))
    
    sim_scores = sorted(sim_scores, key= lambda x:x[1], reverse=True)
    
    sim_scores = sim_scores[0:num+1]
    
    recipe_indices = [i[0] for i in sim_scores]
    
    df_score= sim_scores
    df_cosine = pd.DataFrame(sample_df[['id','name','ingredients']].iloc[recipe_indices],columns =['id','name','ingredients'])
    
    return df_cosine

In [14]:
# Pearson coeficient
tfidf_matrix_array = tfidf_matrix.toarray()
def recommend_pearson(id, num):
    ind = indices[id]
    correlation = []
    for i in range(len(tfidf_matrix_array)):
        correlation.append(pearsonr(tfidf_matrix_array[ind], tfidf_matrix_array[i])[0])
    correlation = list(enumerate(correlation))
    sorted_corr = sorted(correlation, reverse=True, key=lambda x: x[1])[0:num+1]
    recipe_index = [i[0] for i in sorted_corr]
    return sample_df.iloc[recipe_index]

In [15]:
#inddict = indices.to_dict()
#changing the selection of books from index to isbn
#inddict = dict((v,k) for k,v in inddict.items())

## Euclidean Distance function
D = euclidean_distances(tfidf_matrix)

def recommend_euclidean_distance(id,num):
    ind = indices[id]
    distance = list(enumerate(D[ind]))
    distance = sorted(distance, key=lambda x: x[1])
    distance = distance[1:num+1]
    #Get the books index
    recipe_index = [i[0] for i in distance]

    df_euclidean = pd.DataFrame(sample_df[['id','name','ingredients']].iloc[recipe_index],columns =['id','name','ingredients'])
    return df_euclidean

## Test Data
###Test food Ids are - 101, 201, 301,400

In [25]:
get_recommendations_cosine(301,12)

,id,name,ingredients
23,301,asian noodle bowl with sesame salmon,"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
24,302,asian noodle bowl with sesame salmon(Same ingredients),"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
25,303,asian noodle bowl with sesame salmon(Same ingredients with shuffled order),"['roma tomatoes','cucumber','sesame seeds', 'vermicelli', 'rice wine vinegar','skinless salmon fillet' ,'red onion', 'green onion', 'carrots', 'teriyaki sauce','cilantro']"
27,305,asian noodle bowl with sesame salmon(red+green onion replaced by onion only),"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
26,304,asian noodle bowl with sesame salmon(Same ingredients with salmon replaced by chicken),"['chicken', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
29,307,"asian noodle bowl with sesame salmon(removed salmon filet, roma tomtoes, teriyaki sauce,cilantro)","['sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes','carrots',]"
28,306,"asian noodle bowl with sesame salmon(removed teriyaki sauce', 'sesame seeds'+replaced red & green onion with oninon)","['skinless salmon fillet','vermicelli', 'rice wine vinegar', 'onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
30,308,asian noodle bowl with sesame salmon(Same ingredients with shuffled order+ cucumber removed+cilantro and carrots replaced by turmeric& pepper),"['chicken','roma tomatoes','sesame seeds', 'vermicelli', 'wine vinegar', 'green onion', 'pepper', 'teriyaki sauce','turmeric']"
9591,476645,asian slaw salad,"['purple cabbage', 'savoy cabbage', 'edamame', 'carrots', 'cilantro', 'sesame', 'rice wine vinegar', 'splenda sugar substitute']"
7001,308481,asian sesame sauce,"['low sodium soy sauce', 'rice wine vinegar', 'sesame oil', 'garlic', 'sesame seeds', 'green onion']"


In [24]:
recommend_euclidean_distance(301,12)

,id,name,ingredients
24,302,asian noodle bowl with sesame salmon(Same ingredients),"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
25,303,asian noodle bowl with sesame salmon(Same ingredients with shuffled order),"['roma tomatoes','cucumber','sesame seeds', 'vermicelli', 'rice wine vinegar','skinless salmon fillet' ,'red onion', 'green onion', 'carrots', 'teriyaki sauce','cilantro']"
27,305,asian noodle bowl with sesame salmon(red+green onion replaced by onion only),"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
26,304,asian noodle bowl with sesame salmon(Same ingredients with salmon replaced by chicken),"['chicken', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
29,307,"asian noodle bowl with sesame salmon(removed salmon filet, roma tomtoes, teriyaki sauce,cilantro)","['sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes','carrots',]"
28,306,"asian noodle bowl with sesame salmon(removed teriyaki sauce', 'sesame seeds'+replaced red & green onion with oninon)","['skinless salmon fillet','vermicelli', 'rice wine vinegar', 'onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
30,308,asian noodle bowl with sesame salmon(Same ingredients with shuffled order+ cucumber removed+cilantro and carrots replaced by turmeric& pepper),"['chicken','roma tomatoes','sesame seeds', 'vermicelli', 'wine vinegar', 'green onion', 'pepper', 'teriyaki sauce','turmeric']"
9591,476645,asian slaw salad,"['purple cabbage', 'savoy cabbage', 'edamame', 'carrots', 'cilantro', 'sesame', 'rice wine vinegar', 'splenda sugar substitute']"
7001,308481,asian sesame sauce,"['low sodium soy sauce', 'rice wine vinegar', 'sesame oil', 'garlic', 'sesame seeds', 'green onion']"
6918,305013,asian style slaw,"['savoy cabbage', 'radicchio', 'mayonnaise', 'pineapple chunk', 'fresh cilantro', 'red onion', 'rice wine vinegar', 'salt']"


In [27]:
recommend_pearson(301,12)

C:\Users\iPric\anaconda3\lib\site-packages\scipy\stats\stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,name,id,ingredients
23,asian noodle bowl with sesame salmon,301,"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
24,asian noodle bowl with sesame salmon(Same ingredients),302,"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
25,asian noodle bowl with sesame salmon(Same ingredients with shuffled order),303,"['roma tomatoes','cucumber','sesame seeds', 'vermicelli', 'rice wine vinegar','skinless salmon fillet' ,'red onion', 'green onion', 'carrots', 'teriyaki sauce','cilantro']"
27,asian noodle bowl with sesame salmon(red+green onion replaced by onion only),305,"['skinless salmon fillet', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
26,asian noodle bowl with sesame salmon(Same ingredients with salmon replaced by chicken),304,"['chicken', 'teriyaki sauce', 'sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
29,"asian noodle bowl with sesame salmon(removed salmon filet, roma tomtoes, teriyaki sauce,cilantro)",307,"['sesame seeds', 'vermicelli', 'rice wine vinegar', 'red onion', 'green onion', 'roma tomatoes','carrots',]"
28,"asian noodle bowl with sesame salmon(removed teriyaki sauce', 'sesame seeds'+replaced red & green onion with oninon)",306,"['skinless salmon fillet','vermicelli', 'rice wine vinegar', 'onion', 'roma tomatoes', 'cucumber', 'carrots', 'cilantro']"
30,asian noodle bowl with sesame salmon(Same ingredients with shuffled order+ cucumber removed+cilantro and carrots replaced by turmeric& pepper),308,"['chicken','roma tomatoes','sesame seeds', 'vermicelli', 'wine vinegar', 'green onion', 'pepper', 'teriyaki sauce','turmeric']"
7001,asian sesame sauce,308481,"['low sodium soy sauce', 'rice wine vinegar', 'sesame oil', 'garlic', 'sesame seeds', 'green onion']"
1329,asian shrimp salad,53422,"['shrimp', 'soba noodles', 'snow peas', 'red bell pepper', 'carrot', 'onion', 'sweet chili sauce', 'hoisin sauce', 'lime juice', 'cilantro', 'black sesame seed', 'sesame oil', 'rice wine vinegar', 'salt and pepper', 'green onion']"


In [ ]:
D = euclidean_distances(tfidf_matrix)

def jacc_similarity(id, num): 
    jacc_num = 0 
    jacc_den = 0 
    ind=indices[id]
    for index in enumerate(ind): 
        if vector1[index] != 0 or vector2[index] != 0: 
            jacc_den += max(vector1[index], vector2[index]) 
            jacc_num += min(vector1[index], vector2[index]) 
    return jacc_num / jacc_den 